<a href="https://colab.research.google.com/github/charletsebeth/pythontrain/blob/master/Copy_of_Copy_of_Copy_of_Copy_of_Copy_of_XenteV1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

#Visual libaries
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn import svm

**`Export file`**

To be used later

In [0]:
#from google.colab import files
#exportTest = training
#exportTest.to_csv('exportTest.csv') 
#files.download('exportTest.csv')

**How to import files from G-Drive**

In [0]:
# Run this cell to mount your Google Drive.
#Credit given to Medium article
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
sample_submission = pd.read_csv("/content/drive/My Drive/Colab Notebooks/zindi/Xente/data/sample_submission.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/zindi/Xente/data/test.csv")
training = pd.read_csv("/content/drive/My Drive/Colab Notebooks/zindi/Xente/data/training.csv")
Xente_Variable_Definitions = pd.read_csv("/content/drive/My Drive/Colab Notebooks/zindi/Xente/data/Xente_Variable_Definitions.csv")

In [18]:
training.head()



,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [149]:
print(len(test.columns))
print(len(training.columns))

15
16


**Investigate data fields**

View fields and decide what cleaning and feature engineering should be done

In [97]:
Xente_Variable_Definitions

,Column Name,Definition
0,TransactionId,Unique �transaction identifier on platform
1,BatchId,Unique number assigned to a batch of transacti...
2,AccountId,Unique number identifying the customer on plat...
3,SubscriptionId,Unique number identifying the customer subscri...
4,CustomerId,Unique identifier attached to Account
5,CurrencyCode,Country currency
6,CountryCode,Numerical geographical code of country
7,ProviderId,Source provider of Item �bought.
8,ProductId,Item name being bought.
9,ProductCategory,ProductIds are organized into these broader pr...


In [0]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
TransactionId           95662 non-null object
BatchId                 95662 non-null object
AccountId               95662 non-null object
SubscriptionId          95662 non-null object
CustomerId              95662 non-null object
CurrencyCode            95662 non-null object
CountryCode             95662 non-null int64
ProviderId              95662 non-null object
ProductId               95662 non-null object
ProductCategory         95662 non-null object
ChannelId               95662 non-null object
Amount                  95662 non-null float64
Value                   95662 non-null int64
TransactionStartTime    95662 non-null object
PricingStrategy         95662 non-null int64
FraudResult             95662 non-null int64
dtypes: float64(1), int64(4), object(11)
memory usage: 11.7+ MB


In [0]:
training.dtypes

TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
FraudResult               int64
dtype: object

Inspect column types

**Field cleaning and engineering roadmap**

Planning to update below "Roadmap" as the "IF" statements gets answered in my investigation

**Dropping fields**

If I decide to drop a field I would do it near the end of the preprocessing.


**0-TransactionId	:** 
Remove "TransactionId_" and consider dropping field
<br>

**1-BatchId:**
Remove prefix and consider dropping field
<br>

**2-AccountId:**
Remove prefix and consider dropping field
<br>

**3-SubscriptionId:**
Remove prefix and KEEP FIELD, since seems like services that user subscribes too which characterizes person
<br>

**4-CustomerId:**
Remove prefix and CONSIDER dropping field UNLESS repeated fraught offender in dataset
<br>

**5-CurrencyCode:**
Investigate if entries for whole field are the same, IF the same DROP, ELSE make DUMMY

Value is the same across the whole column, therefore drop field
<br>

**6-CountryCode:**
Investigate if entries for whole field are the same, IF the same DROP, ELSE make DUMMY

Value is the same across the whole column, therefore drop field
<br>

**7-ProviderId:** 
Investigate if entries for whole field are the same, IF the same DROP, ELSE make DUMMY
<br>

**8-ProductId:**
Remove prefex
Investigate if entries for whole field are the same, IF the same DROP, ELSE make DUMMY
<br>

**9-ProductCategory:**
Make field dummy variable
<br>

**10-ChannelId:**
<br>
Make field dummy variable

**11-Amount:**
Consider treating positive and negative amounts differently
<br>

**12-Value:**
Consider dropping field, since information is already contained in Amount field
<br>

**13-TransactionStartTime:**
Clean field to make it a "dateTime" field.
extract components of dates into its own fields (e.g. year, day etc), but ignore seconds
Google whether date and time fields should be treated as dummy/categorical features (but might end up with too many fields)
<br>

**14-PricingStrategy:**
Investigate if entries for whole field are the same, IF the same DROP, ELSE make DUMMY

The field is NOT THE SAME across column, keep and make dummy
<br>

**15-FraudResult:**
It is our target variable
<br>


# **Remove repeated shared string in columns**

Quite a few of the fields have some fields that are shared across all of the fields, which I would like to remove.

Therefore need to do two things:

1- Check that assumption that field is shared across all fields are indeed true

2-Create function te remove these fields and apply it

## **Check that assumption that field is shared across all fields are indeed true**

In [0]:
print(training[training['CurrencyCode'] == 'UGX']['CurrencyCode'].count())
print(training['CurrencyCode'].count())

#Not as smart as you people, have to first hardcode, before I can make it a function
bol = training[training['CurrencyCode'] == 'UGX']['CurrencyCode'].count() == training['CurrencyCode'].count()
  
print(bol)

95662
95662
True


In [0]:
#Function not doing what I want, will to check without using the function
def uniqueTest(df,field,dtype,val):
  if dtype == 'string':
    val = str(val)
  else:
      val = int(val)
      
  bol = df[df[field] == val].count() == training[field].count()
  
  return bol

#print(uniqueTest(training,'CurrencyCode','string','UGX'))

In [0]:
#train_df['testBatchID'] = train_df['BatchId'].apply(lambda x: False if x[0:8] =="BatchId_" else True)

def fieldCheck(df,field,word):


  cut = len(word)
  check = pd.DataFrame()
  check['Field'] = df[field].apply(lambda c : c[:cut])
  check['Field']

  check['bol'] = check['Field'].apply(lambda x: True if x != word else False)
  
  #return check
  return check['bol'].sum()

### Text field uniqueness
ish, was stupid, cannot use my functiont to test if prefixes are unique. Will come back at end of sprint if still have time, but will make assumption for now prefixes are unique

**CurrencyCode,  CountryCode, PricingStrategy**

Some fields I can simply see in the df.head that they are not unique, such as:

productID, ProviderId etc

In [0]:
print(training[training['CurrencyCode'] == 'UGX']['CurrencyCode'].count())
print(training['CurrencyCode'].count())

95662
95662


In [0]:
print(training[training['CountryCode'] == 256]['CountryCode'].count())
print(training['CountryCode'].count())

95662
95662


In [0]:
print(training[training['PricingStrategy'] == 2]['PricingStrategy'].count())
print(training['PricingStrategy'].count())

79848
95662


Conclusion:

Drop:

Currency

Country

Keep: Pricing strategy and make dummy

## Create function te remove these fields and apply it

Quite a few of the fields have some fields that are shared across all of the fields, which I would like to remove.

In [0]:
def transColClean(df,columnName,word):
  df[columnName] = df[columnName].apply(lambda s : s.replace(word,''))
  df[columnName] = df[columnName].astype('int64')
  

  return df

Come back later to make sure that prefix is indeed unique if I have time

In [186]:
transColClean(training,'TransactionId','TransactionId_')
transColClean(test,'TransactionId','TransactionId_')

transColClean(training,'BatchId','BatchId_')
transColClean(test,'BatchId','BatchId_')

transColClean(training,'AccountId','AccountId_')
transColClean(test,'AccountId','AccountId_')

transColClean(training,'SubscriptionId','SubscriptionId_')
transColClean(test,'SubscriptionId','SubscriptionId_')

transColClean(training,'CustomerId','CustomerId_')
transColClean(test,'CustomerId','CustomerId_')

transColClean(training,'ProviderId','ProviderId_')
transColClean(test,'ProviderId','ProviderId_')

transColClean(training,'ProductId','ProductId_')
transColClean(test,'ProductId','ProductId_')

transColClean(training,'ChannelId','ChannelId_')
transColClean(test,'ChannelId','ChannelId_')


test.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,50600,35028,2441,4426,2857,UGX,256,5,3,airtime,3,1000.0,1000,2019-02-13T10:01:40Z,4
1,95109,45139,3439,2643,3874,UGX,256,5,15,financial_services,3,2000.0,2000,2019-02-13T10:02:12Z,2
2,47357,74887,4841,3829,2857,UGX,256,4,6,financial_services,2,-50.0,50,2019-02-13T10:02:30Z,2
3,28185,11025,2685,4626,3105,UGX,256,5,10,airtime,3,3000.0,3000,2019-02-13T10:02:38Z,4
4,22140,29804,4841,3829,3105,UGX,256,4,6,financial_services,2,-60.0,60,2019-02-13T10:02:58Z,2


In [166]:
print(len(test.columns))
print(len(training.columns))

15
16


# Time and date fields

>Convert 	"TransactionStartTime" to pandas dateTime dtype

>Break field up into seperate constituents

Decide if time should be a dummy variable (think it will result it too many new columns though)

**Convert "TransactionStartTime" to pandas dateTime dtype**

>Remove the "T" and replace wih space

>Remove "Z"

>Amend field to dateTime

In [0]:
def dateTimeTransform(df):
  df['TransactionStartTime'] = df['TransactionStartTime'].apply(lambda s : s.replace('T',' '))
  df['TransactionStartTime'] = df['TransactionStartTime'].apply(lambda s : s.replace('Z',' '))
  df['TransactionStartTime'] =pd.to_datetime(df['TransactionStartTime'])
  

  return df

**Apply function to training  and test data and check head**

In [188]:
dateTimeTransform(training)
dateTimeTransform(test)
training.head()
#training['TransactionStartTime']

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15 02:18:49,2,0
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15 02:19:08,2,0
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15 02:44:21,2,0
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15 03:32:55,2,0
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15 03:34:21,2,0


In [87]:
print(len(test.columns))
print(len(training.columns))

15
16


## Split date time fields

In [0]:
def dtSplit(df):
  df['Year'] = df['TransactionStartTime'].dt.year
  df['Month'] = df['TransactionStartTime'].dt.month
  #df['Date'] = df['TransactionStartTime'].dt.date
  df['Day'] = df['TransactionStartTime'].dt.day
  df['Hour'] = df['TransactionStartTime'].dt.hour
  df['Minute'] = df['TransactionStartTime'].dt.minute
  df['Dayofweek'] = df['TransactionStartTime'].dt.dayofweek
  
  #training.head()


**Apply function**

In [0]:
dtSplit(training)
dtSplit(test)

In [105]:
test.head(2)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,Year,Month,Day,Hour,Minute,Dayofweek
0,50600,35028,2441,4426,2857,UGX,256,5,3,airtime,3,1000.0,1000,2019-02-13 10:01:40,4,2019,2,13,10,1,2
1,95109,45139,3439,2643,3874,UGX,256,5,15,financial_services,3,2000.0,2000,2019-02-13 10:02:12,2,2019,2,13,10,2,2


In [190]:
print(len(test.columns))
print(len(training.columns))

21
22


# Remove columns not interested in



>TransactionId: drop

>BatchId: Keep and drop
<br>
 Keep since batch is likely a collection of transactions and maybe multiple fraudelent a transactions are executed together. If this is the case it will help classification algorithm, but will have no predictive power outside of sample. (then try again with column being dropped.)

>AccountId: keep, once account has been accessed fraud can occur multiple times
 
>SubscriptionId: drop
<br>
Certain subscriptions might be more prone to fraud, but then I can also simply use ProductCategory or ProductId

>CustomerId: Keep
<br>
Maybe certain customers are easier targets

>CurrencyCode: drop, whole field contains same value

>CountryCode: drop, whole field contains same value

>ProviderId: Keep

>ProductCategory: drop 
<br>
Duplicates ProductId info

>ChannelId: Keep
<br>
Might duplicate ProductCategory, but keep for now

>Value: Drop
<br>
Duplicates amount without stating if positive or negative

>TransactionStartTime: drop, have splitted this field

>PricingStrategy: Values variesin column, will therefore keep


In [0]:
#training.drop(training['TransactionId'], axis=0, inplace=True)
#training.head(1)
# pandas drop columns using list of column names
#gapminder_ocean.drop(['pop', 'gdpPercap', 'continent'], axis=1)
#training.drop(['TransactionId'],axis=1, inplace=True)

#training['TransactionId'].head(2)

In [191]:
#training.drop(['TransactionId'],axis=1, inplace=True)
#fields = ['TransactionId','SubscriptionId','CustomerId','CurrencyCode','CountryCode','ProductCategory','Value','TransactionStartTime']
#test.drop(test[test[fields]], axis=1, inplace=True)

training.drop(['TransactionId','SubscriptionId','CustomerId','CurrencyCode','CountryCode','ProductCategory','Value','TransactionStartTime'], axis=1, inplace=True)
test.drop(['TransactionId','SubscriptionId','CustomerId','CurrencyCode','CountryCode','ProductCategory','Value','TransactionStartTime'], axis=1, inplace=True)


test.head()

,BatchId,AccountId,ProviderId,ProductId,ChannelId,Amount,PricingStrategy,Year,Month,Day,Hour,Minute,Dayofweek
0,35028,2441,5,3,3,1000.0,4,2019,2,13,10,1,2
1,45139,3439,5,15,3,2000.0,2,2019,2,13,10,2,2
2,74887,4841,4,6,2,-50.0,2,2019,2,13,10,2,2
3,11025,2685,5,10,3,3000.0,4,2019,2,13,10,2,2
4,29804,4841,4,6,2,-60.0,2,2019,2,13,10,2,2


In [192]:
print(len(test.columns))
print(len(training.columns))

13
14


# Categorical/Dummy variables

**The easy ones**

Following fields I feel should be dummy variables:

-ProviderId
<br>
-ProductId
<br>
-ChannelId
<br>
-PricingStrategy


**The one's I am unsure of**

This is one of the sections I am most unsure of.

BatchId, AccountId and possibly time are all non numerical data, but to make them all dummy variables might lead to a massive dataframe

So I think I will not make them dummy variables and even consider dropping BatchId, AccountId.

Regarding date and time, I might play around making some of them dummy variables.

In [193]:
ntrain = training.shape[0]
ntest = test.shape[0]

Y = training['FraudResult']
training = training.drop('FraudResult', axis=1)

print("test col count: ",len(test.columns))
print("training col count: ",len(training.columns))

all_data = pd.concat((training, test)).reset_index(drop=True)
#all_data.drop(['FraudResult'], axis=1, inplace=True)

print("all_data col count: ",len(all_data.columns))

test col count:  13
training col count:  13
all_data col count:  13


In [0]:
melist = ['ProviderId', 'ProductId', 'ChannelId','PricingStrategy']
#df_dummies = pd.get_dummies(df, columns = ['ProviderId', 'ProductId', 'ChannelId','PricingStrategy'], drop_first = True)

all_data = pd.get_dummies(training, columns = melist, drop_first = True)
#test = pd.get_dummies(test, columns = melist, drop_first = True)

In [174]:
all_data.head()

,BatchId,AccountId,Amount,Year,Month,Day,Hour,Minute,Dayofweek,ProviderId_2,ProviderId_3,ProviderId_4,ProviderId_5,ProviderId_6,ProductId_2,ProductId_3,ProductId_4,ProductId_5,ProductId_6,ProductId_7,ProductId_8,ProductId_9,ProductId_10,ProductId_11,ProductId_12,ProductId_13,ProductId_14,ProductId_15,ProductId_16,ProductId_19,ProductId_20,ProductId_21,ProductId_22,ProductId_23,ProductId_24,ProductId_27,ChannelId_2,ChannelId_3,ChannelId_5,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4
0,36123,3957,1000.0,2018,11,15,2,18,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,15642,4841,-20.0,2018,11,15,2,19,3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,53941,4229,500.0,2018,11,15,2,44,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,102363,648,20000.0,2018,11,15,3,32,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
4,38780,4841,-644.0,2018,11,15,3,34,3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [195]:
print(len(all_data.columns))


42


# My own train / test split

I will create a train / test split from the train set to fit models to and test accucracy



In [0]:
training = all_data[:ntrain]
test = all_data[ntrain:]

In [197]:
X = training
#X['FraudResult'].head()

print("X col count: ",len(X.columns))
print("Test col count: ",len(test.columns))

X col count:  42
Test col count:  42


In [0]:
#train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.2, random_state = 42) 

In [199]:
print(len(X_train))
print(len(y_train))

76529
76529


In [200]:
X_train.head()

,BatchId,AccountId,Amount,Year,Month,Day,Hour,Minute,Dayofweek,ProviderId_2,ProviderId_3,ProviderId_4,ProviderId_5,ProviderId_6,ProductId_2,ProductId_3,ProductId_4,ProductId_5,ProductId_6,ProductId_7,ProductId_8,ProductId_9,ProductId_10,ProductId_11,ProductId_12,ProductId_13,ProductId_14,ProductId_15,ProductId_16,ProductId_19,ProductId_20,ProductId_21,ProductId_22,ProductId_23,ProductId_24,ProductId_27,ChannelId_2,ChannelId_3,ChannelId_5,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4
59096,5730,4841,-100.0,2019,1,13,8,21,6,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
35991,3209,1270,5000.0,2018,12,21,16,18,4,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
69457,127344,1796,1000.0,2019,1,23,18,29,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
58498,13153,4841,-70.0,2019,1,12,11,14,5,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
88790,15503,2659,20000.0,2019,2,8,7,10,4,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


# Modeling

## Logistic Regression

So I will move between fitting and then making changes to dataset and fitting again.

I am getting a very good fit, so I am  suspecting that the BatchID  or AccountID or something is predicting the results very well, but in the in the real test set it will not be the case
<br>
OR BECAUSE the dataset is so unbalanced, predicting everything is non-fraudelent will get you a good result


In [205]:
clf = LogisticRegression() #(C=30)
#clf.fit(X_train, y_train)
clf.fit(X, Y)

trainTestPredict = clf.predict(X_test)

print(np.mean(trainTestPredict == y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9983797627136362


In [182]:
trainTestPredict

array([0, 0, 0, ..., 0, 0, 0])

# **Preparing output file**



## Generate predictions

In [206]:
print("X_train len: ",len(X_train.columns))
print("X_train shape: ",len(X_train.shape))
print("test len: ",len(test.columns))
print("Test shape: ",len(test.shape))

X_train len:  42
X_train shape:  2
test len:  42
Test shape:  2


In [207]:
predictSubmit = clf.predict(test)

ValueError: ignored

In [141]:
predictSubmit

NameError: ignored